In [ ]:
import pandas as pd
from google.colab import files

# Upload the Excel file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]


Saving NIFTY 50 - Yahoo Finance.xlsx to NIFTY 50 - Yahoo Finance (1).xlsx


In [ ]:
# Load Excel (check structure if there are header issues)
df_raw = pd.read_excel(file_name)

# Clean column names
df_raw.columns = [str(col).strip() for col in df_raw.columns]

# Drop fully empty columns
df = df_raw.dropna(how='all', axis=1)

# Drop rows missing Date or Close
df = df.dropna(subset=['Date', 'Close'])

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Clean and convert numeric columns (remove spaces)
for col in ['Open', 'High', 'Low', 'Close', 'Adj Close']:
    df[col] = df[col].astype(str).str.replace(' ', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Final cleanup
df = df.dropna(subset=['Date', 'Close'])
df = df.sort_values('Date')
df.set_index('Date', inplace=True)

# Check cleaned data
df.head()



,Open,High,Low,Close,Adj Close,Volume,Unnamed: 7
Date,,,,,,,
2020-02-12,12151.00,12231.75,12144.30,12201.20,12201.20,411700,NaN
2020-02-13,12219.55,12225.65,12139.80,12174.65,12174.65,501200,NaN
2020-02-14,12190.15,12246.70,12091.20,12113.45,12113.45,622800,NaN
2020-02-17,12131.80,12159.60,12037.00,12045.80,12045.80,-,NaN
2020-02-18,12028.25,12030.75,11908.05,11992.50,11992.50,676900,NaN


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'))
fig.update_layout(title='📈 NIFTY 50 Closing Price Over Time',
                  xaxis_title='Date', yaxis_title='Price (INR)',
                  template='plotly_dark')
fig.show()


In [ ]:
df['MA50'] = df['Close'].rolling(window=50).mean()
df['MA200'] = df['Close'].rolling(window=200).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close'))
fig.add_trace(go.Scatter(x=df.index, y=df['MA50'], name='MA50'))
fig.add_trace(go.Scatter(x=df.index, y=df['MA200'], name='MA200'))

fig.update_layout(title='NIFTY 50 with Moving Averages',
                  xaxis_title='Date', yaxis_title='Price',
                  template='plotly_white')
fig.show()

In [ ]:
# Create target column - next day's price
future_days = 1
data = df[['Close']].copy()
data['Prediction'] = data['Close'].shift(-future_days)

# Drop missing values
data.dropna(inplace=True)

# Features and labels
X = data[['Close']].values
y = data['Prediction'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_test)

# Evaluate
print("R² Score:", r2_score(y_test, predictions))
print("MSE:", mean_squared_error(y_test, predictions))


R² Score: -0.0062240426305311125
MSE: 48438472214.73154


In [ ]:
import plotly.express as px
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
fig = px.line(comparison, title='📊 Actual vs Predicted Close Prices')
fig.show()


In [ ]:
# Predict the next day
latest = np.array(df[['Close']].tail(1))
next_day_prediction = model.predict(latest)
print(f" Predicted Close Price for next day: ₹{next_day_prediction[0]:.2f}")


🔮 Predicted Close Price for next day: ₹24183.26


In [ ]:
# dev 3 models
from sklearn.model_selection import train_test_split

# Create new feature-target DataFrame
future_days = 1
data = df[['Close']].copy()
data['Prediction'] = data['Close'].shift(-future_days)
data.dropna(inplace=True)

# Define features (X) and target (y)
X = data[['Close']].values
y = data['Prediction'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Define and train models
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    r2 = r2_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)

    results[name] = {
        "model": model,
        "predictions": predictions,
        "r2": r2,
        "mse": mse
    }

    print(f"📊 {name}")
    print(f"   R² Score: {r2:.4f}")
    print(f"   MSE     : {mse:.2f}")


📊 Linear Regression
   R² Score: -0.0062
   MSE     : 48438472214.73
📊 Random Forest
   R² Score: -0.1793
   MSE     : 56768696286.00
📊 XGBoost
   R² Score: -0.0649
   MSE     : 51262349382.73


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
actual = y_test[:100]

# Add model predictions
for name, result in results.items():
    fig.add_trace(go.Scatter(y=result["predictions"][:100], mode='lines', name=name))

# Add actual data
fig.add_trace(go.Scatter(y=actual, mode='lines', name='Actual', line=dict(color='black', dash='dash')))

fig.update_layout(title="📊 Actual vs Predicted Prices (100 Samples)",
                  yaxis_title="Price", template="plotly_white")
fig.show()


In [ ]:
#Predict Tomorrow’s Price from Last Known Close
latest_input = np.array(df[['Close']].tail(1)).reshape(1, -1)

for name, result in results.items():
    tomorrow_price = result['model'].predict(latest_input)[0]
    print(f"🔮 {name} predicts tomorrow’s price: ₹{tomorrow_price:.2f}")


🔮 Linear Regression predicts tomorrow’s price: ₹24183.26
🔮 Random Forest predicts tomorrow’s price: ₹23150.26
🔮 XGBoost predicts tomorrow’s price: ₹23167.88


In [ ]:
# Compare All Model Performances
comparison_df = pd.DataFrame({
    "Model": list(results.keys()),
    "R² Score": [results[m]['r2'] for m in results],
    "MSE": [results[m]['mse'] for m in results]
})
comparison_df


,Model,R² Score,MSE
0,Linear Regression,-0.006224,4.843847e+10
1,Random Forest,-0.179270,5.676870e+10
2,XGBoost,-0.064885,5.126235e+10
